**Implementation and evaluation of region growing**

Data Analysis MoBi SS2021, Topic 04: Biomedical image analysis

Tutor: Nicolas Peschke

Group 04: Marie Becker, Ina Jung, Laura Kaschnitz, Johanna Möller

<br>

**Abstract**

Region growing is an interesting approach for image segmentation which can be used to evaluate microscopy images and detecting cell structures.
It is further divided it into seeded and unseeded region growing. <br>
In this project, algorithms for both seeded and unseeded region growing are implemented and executed on the datasets N2DH-GOWT1, N2DL-HeLa and NIH3T3.
Preprocessing and parameter adjustment for merging is applied for difficult datasets to reduce noise and focus on the important information.
The dice score evaluates the result and the performance of the algorithm by comparing the segmented images to a manually segmented ground truth image. <br>
Both seeded and unseeded region growing algorithms lead to very good results with high dice scores up to 0.99 for simple segmentation tasks.
The vectorization of the algorithm allows acceptable speeds despite the large computing power required, since each pixel is assigned individually and intermediate steps in the algorithm are permanently updated.
A simultaneous allocation of the pixels could lead to equally good results and faster runtime.
Interfering image elements with strongly deviating pixel intensities compared to their neighborhood are still a challenge to be overcome.
Even after applying the algorithm, some pixels will not be segmented properly and must be added by manual parameter determination. <br>
With an improvement of the runtime by simultaneous assignment of the pixels and automatic parameter determination in combination with better preprocessing, these challenges could be mastered and a more robust algorithm could be obtained.


<br>

**Table of content**

1.	Introduction
2.	Datasets – N2DH-GOWT1, N2DL-HeLa and NIH3T3
3.	Imports
4.	Preprocessing <br>
4.1 Clipping <br>
4.2 Filtering <br>
5.	Segmentation algorithm <br>
5.1 Seeded region growing algorithm <br>
5.1.1 Seed selection <br>
5.1.1.1 Seed selection based on similarities <br>
5.1.1.2 Seed selection based on Eucledian distance <br>
5.1.2 Seeded region growing <br>
5.2 Unseeded region growing algorithm <br>
5.3 Region merging <br>
5.3.1 Region merging based on similarity of intensity <br>
5.3.2 Region merging based on size <br>
5.4 Differences between seeded and unseeded region growing <br>
6.	Evaluation measures <br>
6.1 Dice score <br>
7. Results <br>
7.1 N2DH-GOWT1 <br>
7.2 N2DL-HeLa <br>
7.2.1 Shifted intensity range in N2DL-HeLa <br>
7.2.2 High cell density and intensity differences of nuclei <br>
7.2.3 Final results <br>
7.3 NIH3T3 <br>
7.3.1 Bright spots in NIH3T3 data set <br>
7.3.2 Blurs in NIH3T3 data set <br>
7.3.3 Changing background intensities <br>
7.3.4 Final results <br>
7.4 Evaluation of preprocessing <br>
7.5 Comparison Seeded and unseeded region growing results <br>
8. Challenges <br>
8.1 Optimizing Seeded region growing algorithm <br>
8.2 Optimizing Unseeded region growing algorithm <br>
8.3 Comparison Seeded and Unseeded region growing algorithms <br>
8.4 Memory error <br>
9. Discussion
10. Bibliography

<br>

# Introduction

Image segmentation is an image processing program in the field of data analysis, whose role has become increasingly important in recent years.
It consists in grouping pixels of the image by commonalities of image intensities and thus creating regions of pixels that can be used for data analysis and evaluation.
In this way, image content can be simplified, and its representation can be modified by using methods like thresholding, boundary or region-based segmentation or hybrid techniques.
Image segmentation is often the first step in image analysis. It is widely used, especially in the medical field, but also in the processing of geographical data.

Region growing is a subset of the hybrid techniques of image segmentation which can be used for cell nuclei segmentation.

This class of methods can be further subdivided into seeded and unseeded region growing, while the focus in this project lies on seeded region growing.
Here, selected initial seeds are assigned pixels from the neighborhood if their relative Euclidean distance to the seeds is below a certain threshold.
Subsequently, certain regions are merged to combine similar or small regions (Shih and Cheng, 2005).

After implementing the seeded region growing algorithm, its performance will be evaluated.
For this, result images are compared to already segmented gt-images by using the Dice score.
Finally, seeded and unseeded region growing are also compared with each other using the Dice Score.

<br>

# Datasets – N2DH-GOWT1, N2DL-HeLa and NIH3T3

In order to run and test the implemented seeded region growing algorithm, three different datasets were used: N2DH-GOWT1, N2DL-HeLa and NIH3T3.
The datasets consist of a total of 28 image files, in which different structures of the nucleus were stained using the fluorescent protein GFP.
Attached to the data sets are gt-images where the nuclei have already been segmented.
The images and gt-images of these datasets are often used to evaluate the performance of newly implemented algorithms.

Each dataset has specific challenges that need to be considered.

In the first dataset N2DH-GOWT1, transcription factor Oct 4 was stained by GFP in mouse embryonic stem cells.
The nuclei are well in focus and do not overlap.
There are no reflections or blurs.
However, the stained nuclei differ greatly in brightness.
Compared to the other data sets, N2DH-GOWT1 is considered to be the easiest to work with.

N2DL-HeLa shows GFP labeled core histone 2b proteins from human cervical carcinomas.
Overall, the cell density is high whereas the resolution is low, and the nuclei vary greatly in brightness.
Furthermore, the intensity values of the images lie between 33 000 and 36 000.
There are no interfering effects due to reflections or blurs.

The last dataset NIH3T3 consists of mouse embryo fibroblasts.
EGFP was used to label the CD-tagged protein.
In NIH3T3 the nuclei are displayed in a very low contrast and differ strongly in their size and shape.
Very bright regions, presumably caused by reflections from the microscope, probably make this data set the most challenging.

<br>

# Imports

In order to test the implemented algorithm, initial uploads of the datasets and various modules were made. <br>
The modules access self-implemented functions in other notebooks or import python modules lieke numpy, skimage and medpy.

In [ ]:
%load_ext autoreload
%autoreload 2

import skimage.io as sk
from Functions import image_processing as ip
from Functions import seeded_region_growing as srg
from Functions import unseeded_region_growing as urg
from Functions import seed_detection as sd
from Functions import visualization as vis

The test data sets were uploaded to the GitHub.
Consequently, they can be accessed and displayed on PyCharm.
Here an image of each data set is shown.

In [ ]:
vis.load_images()

# Preprocessing
Preprocessing is an important step in the preparation of data.
Clipping, filtering and other methods can be used to optimize the data to yield better results of the algorithm.
For instance, background noise can be decreased, and the image can be reduced to its essential content.
However, it is crucial to avoid losing too much relevant information by overuse of preprocessing.

The algorithm will be applied to both unprocessed and preprocessed images.
By comparing the result images the influence of preprocessing can be tested.

## Clipping
With the clipping method, all intensity values of an image below or above manually defined thresholds will be set to their value.
This allows to combine extreme values and to reduce the intensity spectrum.

For clipping, a pre-implemented method from the numpy module is used.

In addition, a separate method was implemented in which the values below or above the thresholds were set to the minimum and maximum intensities of the image.
This variant is intended to increase the contrast of the combined intensities to the remaining.
Below the unclipped picture (left), the normal clipped picture (middle) and the picture clipped with the self implemented funtion (right) are shown:

In [ ]:
vis.clipping_examples()

In order to reduce disturbances of the image caused by very bright intensity values, the functions remove_bright_spots
and remove_bright_spots_with_border were implemented. The first method sets the intensity values of bright spots to an
intensity similar to the background.
The second method clips the bright spot together with a border around the bright spots of variable size.
This will be very useful for the third data set to fade very bright spots into the background using two different thresholds:

In [ ]:
vis.bright_spots_example()

## Filtering
Median ang gaussian filtering were considered to improve the segmentation results.

The median filter sorts the intensity values of the specified neighborhood and assigns the median value to the central pixel.

The Gaussian filter is a linear filter that weights the intensity values of the specified neighborhood with a Gaussian distribution.
Subsequently, the calculated mean value is assigned to the central pixel.
Due to the weighting, the Gaussian filter is an improvement of the Mean filter.

A pre-implemented function of the numpy module has been used for both filtering methods.

# Segmentation algorithms
## Seeded region growing algorithm
Seeded region growing is a sub-method of image segmentation.
It consists of the determination of initial seeds, followed by the assignment of pixels to the seeds, known as seeded region growing.
Regions can then be combined in region merging.
The implementation of the seeded region algorithm is based on the paper of Frank Y. Shih and Shouxian Cheng (Shih and Cheng, 2005).

### Seed selection
In order for pixels to be selected as seeds, they must fulfill two conditions.
One concerns the similarity, the other the Euclidean distance of a pixel to its neighborhood.

#### Seed selection based on similarities

To evaluate the similarity, the standard deviation $$\sigma$$ of each pixel to its 3x3 neighborhood is calculated:

$$ \sigma = \sqrt{\left(\frac{1}{9}\sum\limits_{i=1}^{9}(x_i-\overline{x}\right)^2}$$

with $$x_i$$ as pixel intensities and $$\overline{x_i}$$ as the mean intensities of the 3x3 neighborhood of a pixel.
$$\sigma$$ is then normalized by dividing by the maximum value of all calculated standard deviations and the similarity $$s$$ is obtained from the following equation:

$$s=1-\frac{\sigma}{\sigma_{max}}$$
The similarity $$s$$ is then compared to a threshold.
If $$s$$ is greater than the threshold, the pixel can be considered as seed. For this purpose, Otsu thresholding was used to define the threshold value.

#### Seed Selection based on Euclidean Distance

Now all pixels whose similarity $$s$$ is greater than a threshold are further used for seed selection based on Eucledian distance in order to exclude pixels that lie on the border between two regions.
The maximum value $$d_{max}$$ of the Euclidean distances of a pixel to its neighborhood is calculated by:

$$d_{max} = \mathop{max}_{1\leq i\leq 8} \left\{ d_i = \frac{x-x_i}{x} \right\}$$
$$d_{max}$$ is then also compared to the threshold.
If $$d_{max}$$ is lower than the threshold, the pixel is considered a seed.

For visualization the  algorithm is applied to a section of the loaded image (left).
The implemented algorithm results in the following seeds (right).

In [ ]:
image_intensity_small = sk.imread("Data/N2DH-GOWT1/img/t01.tif")[300:400, 400:500]
image_seeds = sd.seeds(image_intensity_small, 0.5)
ip.show_two_images_colorbar(image_intensity_small, image_seeds, 0.65)

### Seeded region growing
Before starting the region growing algorithm based on the selected seeds, the seeds have to be assigned to regions.

In [ ]:
image_regions_from_seeds = sd.seed_merging(image_seeds)

The seeded region growing algorithm labels all pixels to the regions created out of the seeds iteratively.
Thereby, it always labels the pixel with the closest intensity distance to one of its neighboring regions first
(Gómez et al., 2007), (Shih and Cheng, 2005).

The algorithm follows the steps below:
1. First, the neighboring pixels to the created regions are selected and stored in four 2d numpy arrays for left, right,
top and bottom neighbors.
1. The distance of these pixels to its neighboring regions is calculated based on previously computed mean intensity
values of the regions. The formula $$dist = \frac{|p - \bar r_n|}{intensity_{max}}$$ is used.
1. The smallest distance is selected, and the associated pixel position determined. This pixel is now merged to its
neighboring regions.
1. Before the next pixel can be selected, the mean intensity value of the changed region, the arrays of neighboring
regions, and the distances to the changed region need to be recalculated.
1. Again, the most similar pixel is selected and labeled. This procedure is repeated until all pixels are labeled.

Below the results of seeded region growing are shown (original image left, seeds middle, image after region growing right)

In [ ]:
image_regions = srg.region_growing(image_intensity_small, image_regions_from_seeds.copy())
ip.show_three_images_colorbar(image_intensity_small, image_seeds, image_regions, 0.45)

## Unseeded region growing algorithm
Furthermore, another region growing algorithm was implemented based on the paper of Zheng Lin, Jesse Jin and Hugues Talbot (Lin et al., 2000). In difference to seeded region growing where several pixels are detected before the algorithm start, in unseeded region growing only one start pixel is chosen randomly. The algorithm is similar to seeded region growing and follows the steps below:
1. Based on the first seed the neighboring pixel are selected and stored in 2d numpy arrays for left, right top and bottom neighbors.
1. The intensity distance of these neighboring pixels to the start region is calculated according to <br>
$$dist = |p - \bar r_n|$$
1. The smallest distance and the associated pixel are selected.
1. This pixel is now labeled. For labeling three options exist:
1. If the intensity distance of the pixel is smaller than a threshold, the pixel is merged to its neighboring region
1. If the intensity distance of the still unlabeled pixel to another region is smaller than the threshold, the pixel is merged to this region. This only makes sense when the algorithm continues and more regions exist.
1. If both previous conditions are false, the pixel must belong to a new region. A new region is added.
1. After labeling the first pixel, the arrays of neighboring pixel need to be updated, and the intensity distances are recalculated.
1. This is used to calculate the pixel which is labeled next according to the above conditions. The algorithm continues until all pixel are labeled to a region.
The result of unseeded region growing can be seen below (unprocessed image left, result of seeded region growing right).
Here the pixel on position (0,0) is selected as starting point.

In [ ]:
image_result_unseeded = urg.unseeded_region_growing_algorithm(image_intensity_small, (0,0), 5)
ip.show_two_images_colorbar(image_intensity_small, image_result_unseeded, 0.65)

## Region merging
In both algorithms, seeded and unseeded region growing too many regions are received due to intensity differences in the background as well as in the cell nuclei.
To solve this over-segmentation a region merging algorithm was implemented.
The received regions are merged based on two criteria: similarity of intensity values and region size.

### Region merging based on similarity of intensity
The first selection criteria is implemented via the following procedure:
1. First the neighboring regions are determined for every region and stored in a 2d numpy array.
1. This is used to calculate the distances between all neighboring regions which are stored in an upper triangle 2d
numpy array. The distance between two regions is calculated using <br>
$$dist = \frac{\bar r_m - \bar r_n}{intensity_{max}}$$
1. Then the most similar regions are selected and merged.
1. This procedure is repeated until the distance between the most similar regions is bigger than a threshold. For that, in every
iteration the mean intensity value of the newly merged region, and the distances to its neighboring regions are updated.
The numpy array, in which neighboring regions are stored is updated as well.

### Region merging based on size
Region merging by size follows the steps below:
1. First the size of every region is calculated and stored in a 1d array.
1. The smallest region is selected and compared to a threshold. If it is smaller, the algorithm continues.
1. The previously determined neighboring regions are used to find out the most similar neighboring region by calculating
the distances using the same formula. The selected smallest region is merged to its most similar neighbor.
1. Afterwards, region sizes are updated, the smallest region is recomputed and its size is compared to the threshold.
1. The next most similar neighboring region is determined based on recalculated distances and neighboring regions.
The algorithm stops as soon as the smallest region is bigger than the threshold.

To easily differentiate between nuclei and background, a clipping algorithm is used on the result of the region merging
algorithm. This algorithm clips nuclei to the intensity value 1 and background to the value 0.

In [ ]:
vis.region_growing_example()

The figures compare the over-segmented image with the merged image after region merging by intensity
similarity, with the image after complete merging, and with the clipped image (from left to right).
The first row shows the results of seeded region growing.
In the second row the results of unseeded region growing can be seen.

## Differences between Seeded and Unseeded region growing
In seeded region growing the pixels of one region are always connected while in unseeded region growing,
pixels which are assigned to the same region, but are not connected can appear.
This leads to differences in the region merging process. While region merging by intensity distance
leads to good results for seeded region growing, for unseeded region growing pixels of the cell nucleus
are easily merged to the background. Because of that, a small threshold needs to be chosen for region
merging by intensity distance.

Region merging by size leads to better results, although some unconnected pixel remained.
That's because in unseeded region growing regions are not limited to one nucleus.
So single pixel appear which belong to big, distant regions and can not be merged.
This resulted in spots, which resembled salt and pepper noise.

In order to improve our result,
a median filter with a 3x3 sized filter mask was used which improved the dice score. When using a
bigger filter size, the holes within the nuclei shirk, which is an improvement, but nuclei that are
only partly detected are merged to the background and lost. We decided that it is more important to
see all nuclei, even if they are only partly detected. Following this, a filter with size 3 was used.

Another difference to consider is the normalization of calculated intensity distances.
In seeded region growing intensity distances are always normalized to values between 0 and 1.
Instead, in unseeded region growing only differences are calculated. This needs to be considered in preprocessing.
However, in region merging normalized distances are used which also affects unseeded region growing.

# Evaluation measures
## Dice score
The dice score is a method to determine the accuracy of the segmentation. Thereby, a ground truth image (given resulting picture) is compared with
the resulting image of the algorithm. The dice score is defined as follows:
<br>
$$ DSC = \frac{ 2 \times |S \cap G|}{|S| + |G|} $$
<br>
The intersection is the number of pixels that have the same value in the ground truth and result image. The doubled
intersection is divided by the total number of pixels in the ground truth and result image. This delivers a number
between zero and one. The higher the value the better was the performance of the algorithm.
<br>
To evaluate an algorithm more accurately the dice score can be weighting equally between different regions
(weighted dice score).
This makes it possible to consider small and bigger regions equally.
This is important in nuclei detection because there are proportionally few nuclei compared to the background.
So loosing nuclei would not have such a high impact on the unweighted dice score as desired.
<br>
$$\overline{DSC}=\frac{1}{2} \sum\limits_{i=1}^{2} \left(\frac{2 \times |S_i \cap G_i|}{|S_i|+|G_i|}\right)$$
<br>
In this case two regions were considered: nuclei and background. For each region, the dice score was calculated
individually and then weighted equally.
For implementation, the background was defined as the largest region and all other regions were considered as nuclei
in order to create a binary array.
This makes it possible to compare the ground truth, and the resulting images without depending on equal region numbers
for every nucleus.

# Results
## N2DH-GOWT1
The result of the seeded region growing on the image t01 of the N2DH-GOWT1 is shown below
(original image, result of seeded region growing, result after merging and clipping, gt image (from left to right)).
An unweighted dice score of 0.982, and a weighted dice score of 0.928 were obtained.

Comparing the original image with the segmentation result all visible nuclei were caught.
However, in the ground truth image some nuclei are missing, which might be a mistake in the ground truth images.
This difference is reflected in the dice score. In the weighted dice score missing nuclei have a bigger impact.
The weighted dice score result of 0.928 is still satisfying.

In [ ]:
vis.results_gowt1_seeded()

The following images show the results of the unseeded region growing algorithm on the same picture (t01).
A dice score of 0.963 (unweighted) and 0.870 (weighted) was achieved.
The lower weighted dice score confirms the remaining salt and pepper noise remaining in the merged picture (left),
which is also visible in the final clipped image (second picture).
A 3x3 median filter (third image) improves the result, and the clipped image (right).
The weighted dice score increases to 0.929 (unweighted: 0.982) which is similar to the result of seeded region growing.
The similar dice scores, and the same detected nuclei with both algorithms strengthen the theory of mistakes in the ground truth images.

In [ ]:
vis.results_gowt1_unseeded()

## N2DL-HeLa
### Shifted intensity range in N2DL-HeLa
For seed detection in the seeded region growing algorithm an otsu-threshold is used.
Since normalized intensity distances are used, it is necessary to normalize the otsu-threshold.
This works fine in images with an intensity range starting with 0.
In the N2DL-Hela data set though a very high otsu-threshold was obtained due to the normalization on an intensity range
between 33000 and 36000. This lead to memory errors because of too many seeds and resulted in worse algorithm outcomes
after seeded region growing and region merging because of decreased intensity distances. ---> ist das so??
Subtracting the minimal image intensity and thereby shifting the intensity range solves this issue.

In unseeded region growing this issue doesn't occur because intensity distances are not normalized. However, for region
merging by intensity distance very small intensity distances are obtained due to shift of intensity range
which have to be considered in choosing thresholds.

### High cell density and intensity differences of nuclei
Due to the high cell density in this data set parameters for region merging by size have to be adapted.
While for the N2DH-GOWT1 regions smaller than 400 pixel are merged, for N2DL-Hela images only 100 is used as parameter.

Moreover, the nuclei differ greatly in intensity.
This needs to be considered for region merging based on intensities distances. The threshold has to be adjusted to the
darkest nuclei to prevent loosing nuclei.

### Final results

In [ ]:
vis.results_hela()

Seeded and unseeded region growing were performed on the image t13 of the N2DL-Hela Data set.
To improve the results of seeded region growing, the normal image (first image) was preprocessed with the following functions and parameters:
- ip.subtract_minimum: image
- ip.image_clipping: image, 0.05 * np.amax(image), 0.15 * np.amax(image)
- ip.median_filter: image, 3

For seeded region growing (second image), the parameter 0.05 was used for the sd.seeds function and 20 for the sd.reduce_region_number function.
Region merging was performed with parameters of 0.1 and 100. The final result after clipping can be seen in the third image.

For unseeded region (fourth image) growing preprocessing was not performed because good results were obtained without it.
For computing the unseeded region growing algorithm, (0,0) was selected as the start pixel and 50 as the similarity threshold.
In the fifth picture the result after merging with parameters of 0.000001 and 10000, filtering with a 3x3 median filter to remove salt and pepper noise and clipping is shown.

Comparing the resulting images with the gt image (sixth image) shows that in unseeded region growing most nuclei
were detected, while after seeded region growing some nuclei are missing.
The weighted dice scores of the results confirm this. It is noticeable that the first data set lead to better dice scores.

A reason why more nuclei are lost compared to the first data set are the higher cell density,
and less intensity distances between background and nuclei. Unseeded region growing seems to be more robust,
while in seeded region growing seed detection in all nuclei was difficult.


## NIH3T3
### Bright spots in NIH3T3 data set
As mentioned before, the pictures of the third data set contain bright spots due to reflections of the microscope.
This raises the risk of detecting the reflections as nuclei.
Preprocessing to merge the bright spots into the background was evaluated.

Removal of bright spots (middle) and removal of bright spots and a border (right) are compared.
Removing also the border of the bright spots reduces the transition region between the bright spot and the background.
This region can not be removed otherwise because of its similar intensity to the nuclei.

Nevertheless, the risk of merging nuclei to the background is higher when the border are removed along with the bright
spots. The small clipped parts inside the nuclei after removal of the bright spots without a border are removed easily
during region merging by size.
Removing the border as well leads to bigger missing nuclei parts and might make it impossible to compensate the missing
parts in region merging.
That's why it is important to choose a threshold which keeps the nuclei completely intact,
if bright spots and borders are excised.

In [ ]:
vis.nih3t3_show_bright_spots()

The images are now segmented and clipped afterwards to differentiate nuclei and background.
Seeded region growing is used.

In [ ]:
vis.nih3t3_srg_bright_spots()

In the unprocessed image (left) both, bright spot and nuclei are detected as nuclei.
This confirms that a removal of the bright image spots is necessary.
In the image with intact nucleus (middle) only the reflection is detected.
Because too less of the bright spot was removed, adjusting the intensity distance merging parameters
rather merges the nucleus to the background than the reflection.
Removing a border along with the bright parts of the reflection solves this problem.
Because nearly the whole bright spot is removed, it can be merged to the background by intensity distance.

The dice scores confirm that the third image was segmented best.
However, the weighted dice score of the third image reveals, that a part of the nucleus was merged to the background.
This happens because the reflection leads to a smaller intensity difference between background and nucleus.

### Blurs in NIH3T3 data set
Another Challenge that had to be faced is, that some images of the third data set contained blurs, that are
smaller than the nuclei but have similar intensities. They are registered as a region which mean is
relatively close to a region of a nuclei and are therefore merged to the nuclei even if the threshold
for merging by intensity is low. With merging by size, the blurs are merged to the nuclei. This
happens because their intensities are more similar to the nuclei than the background.

In [ ]:
vis.nih3t3_blurs()

### Changing background intensities
In some images of the third data set the background intensities are changing throughout the images.
This makes it difficult to merge the background to one region after segmentation which is necessary to differentiate
between nuclei and background.
Because of that, especially in the lighter parts of the image nuclei are merged to the background.
This has less impact on unseeded region growing because there regions aren't limited to on nucleus. --> eventuell in die andere Richtung argumentieren

### Final results
Below the image dna-42 of the NIH3T3 data set is shown (first image).
Seeded (second image) and unseeded (fourth image) region growing were performed.
For seeded region growing bright spots with border were removed (130,60,30) and a 3x3 gaussian filter was used.
---> Seeds 0.001, 5
---> Merged 0.1, 400

Although the bright spot could be successfully removed in an image fragment
the different background intensities prevented this on the whole image.
--> Add results seeded and unseeded similar to second data set

## Evaluation of preprocessing
To test, which preprocessing method leads to the best result, different clipping methods and filters were compared. The
total algorithm for seeded  and unseeded region growing was applied to a fraction of an image of the second data set. On the first
data set high dice scores were obtained without preprocessing. Using the weighted and unweighted dice
score, the best parameters where defined for every image. Thereby the focus was laid on the optimization of the weighted
dice score.
The first row of the following images shows the different preprocessing methods
(unprocessed, clipped, extreme clipped, median filtered, gauss filtered, gt).

In [ ]:
vis.show_preprocessing()

On these pictures the seeded (second row) and unseeded (third row) region growing algorithm were conducted. The parameters were determined individually
for every preprocessed image.
The obtained weighted dice scores are shown in this bar plot.

In [ ]:
vis.barplot_preprocessing()

It is visible that the algorithm is working well on every image for seeded and unseeded region growing.

After seeded region growing, the weighted dice score after gauss filtering (0.888) is the lowest,
while all other weighted dice scores are above 0.9.
This might be because, the Gauss filter blurs edges and thereby complicates the identification of nuclei.
All other preprocessing methods increase the weighted dice score, but only clipping improves the weighted
dice score significantly (approximately 0.91 to 0.98).
Normal clipping performs best (0.983). Unweighted dice scores conduct similarly.

Unseeded region growing showed similar results.
A high weighted dice score of 0.945 on the unprocessed image was determined
as well as good results on every preprocessed image. All preprocessing methods improved the weighted dice score.
The highest score was obtained with median filtering (0.9839) opposed to seeded region growing, followed by clipping (0.9835).
Compared to seeded region growing unseeded region growing showed better results overall.

## Comparison Seeded and Unseeded region growing results
For both, seeded and unseeded region growing, parameters need to be determined for every image and data set individually.
However, after optimizing preprocessing methods and parameters unseeded region growing lead to better results,
especially in challenging data sets. This can be seen in the bar chart below.

In [ ]:
vis.barplot_results()

# Challenges
## Runtime errors
### Optimizing Seeded region growing algorithm
The paper of Frank Y. Shih and Shouxian Cheng (Shih and Cheng, 2005) on which the used seeded region growing algorithm is based labels every pixel one by one which leads to
long runtimes. This is why many runtime problems were encountered, especially with the first seeded region growing
algorithm implemented.

In the first region growing algorithm all pixels bordering an existing region are stored in a list. For loops are used
to iterate over this list to find the intensity distance of these pixels to its closest neighboring region. It was
observed that with this algorithm its impossible to segment the whole image in a sensible time with the available
hardware.

Hence, a new algorithm was implemented. It was evaluated to use numpy arrays or pandas data frames to vectorize the
algorithm and decided to use numpy because of faster runtimes. Instead of storing only the smallest distance of every
neighboring pixel to one of its adjacent regions, every distance is stored and does not need to be recalculated.
Moreover, the use of for loops was almost completely reduced, and the algorithm was vectorized.
A decrease of runtime by more than 20 times on a part of an
image could be achieved. The improvement on the whole image is even bigger. This allows to execute the region growing
algorithm on a whole image in 24 hours.

In [ ]:
image_intensity_small = sk.imread("Data/N2DH-GOWT1/img/t01.tif")[300:350, 400:450]
#%timeit srg.region_growing(image_intensity_small, image_regions_from_seeds.copy())
#%timeit old_srg.region_growing(image_intensity_small, image_regions_from_seeds.copy())

### Optimizing Unseeded region growing algorithm
The in the paper of Zheng Lin, Jesse Jin and Hugues Talbot (Lin et al., 2000) proposed unseeded region growing algorithm leads to the same problem. Because every pixel is
selected separately, the runtime of the first implemented algorithm is too long to segment a complete image with our
available hardware.

That's why we rewrote the algorithm similarly to the seeded region growing algorithm. The runtime became nearly 100
faster.

In [ ]:
#%timeit urg.unseeded_region_growing_algorithm(image_intensity_small, (0,0), 5)
#%timeit old_urg.old_unseeded_region_growing_algorithm(image_intensity_small, (0,0), 5)

### Comparison of Seeded and unseeded region growing algorithm
Whilst comparing both algorithms, no significant runtime difference between both algorithms can be discovered if
compared on a small image. The runtime difference lies within the $$3 \sigma$$ interval.

In [ ]:
%timeit srg.region_growing(image_intensity_small, image_regions_from_seeds.copy())
%timeit urg.unseeded_region_growing_algorithm(image_intensity_small, (0,0), 5)

## Memory error
In the region merging algorithm an array with the same number of rows and columns as the region number
is needed. If too many seeds are found in the seed detection, too many regions are detected in the seeded region
growing, and the region merging algorithm can not be performed. When trying to do so, a memory error occurs.
That's why a method to reduce the seed number was implemented.

# Discussion
Despite the challenges of the different data sets images of each data set could be segmented successfully
using preprocessing. Especially in the first data set N2DH-GOWT1 satisfying results and high dice scores were obtained.
In the second data set (N2DL-Hela) normalizing and preprocessing could be used to gain high dice scores.

--> final evaluation third data set
Removing the bright spots before segmentation in the third data set (NIH3T3) solved this problem.
However, bright spots in the same intensity range as nuclei can not be removed.
This problem needs to be solved in future research.

Preprocessing, especially clipping, of the images improved the results and is therefore recommended.
For filtering a median filter showed better results compared to gauss because it does not blur edges and should be used.

Although both, seeded and unseeded region growing, lead to good results,
unseeded region growing was more robust against challenging images.
One reason for that is the seed detection in seeded region growing which often fails to detect nuclei
or background regions between nuclei.
Increasing the number of seeds can't solve this problem because it leads to memory errors.
Alongside less memory errors and smaller execution times for region merging, the smaller region number in unseeded
region growing leads to regions with several nuclei. This compensates small intensity distances of nuclei to their
background in bright background regions or because of dark nuclei.
An advantage and disadvantage of unseeded region growing is its smaller number of region.
However, a disadvantage of unseeded region growing are additional filtering methods which need to be applied after region merging.
Since both algorithms still showed good results and dice scores they can be considered in future research.

For evaluation only the dice score was used.
Thereby, the weighted dice score lead to more meaningful results compared to the unweighted dice score because it
considers nuclei and background equally.
Still high weighted dice scores are received even if many nuclei are missing because of the impact of the background.
Differences of ground truth images, and the received results are partly caused
by mistakes in the ground truth images which don't show some nuclei, especially on image borders.
Because of these to problems, other evaluation methods should be considered in future research.

Despite the big runtime improvements, the runtime in both proposed algorithms is still too long. An algorithm which
labels more than one pixel in one iteration could fix this problem. That's why we recommend using another algorithm
of another paper in following research.

All in all, both segmentation lead to satisfactory results with specifically adjusted parameters for each image.
Nevertheless, in future research the process of determining parameters needs to be automated
to make it available for everyone and to gain actual time reduction in nuclei detection via image segmentation.

# Bibliography

1. Lin, Z., Jin, J., and Talbot, H. (2000). Unseeded region growing for 3D image segmentation. In Selected papers from the Pan-Sydney workshop on Visualisation - Volume 2 (Sydney, Australia: Australian Computer Society, Inc.), pp. 31–37.
1. Shih, F.Y., and Cheng, S. (2005). Automatic seeded region growing for color image segmentation. Image and Vision Computing 23, 877-886.
1. Gómez, O., Gonzalez, J., and Morales, E. (2007). Image Segmentation Using Automatic Seeded Region Growing and Instance-Based Learning, Vol 4756.




